<a href="https://colab.research.google.com/github/gmccu73096/PortfolioProjects/blob/main/TripData_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Preparation

In [ ]:
#mount your drive
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
#import libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl

In [ ]:
#Upload dataframe
year_df = pd.read_csv('/content/drive/MyDrive/Data Analysis/tripdata/tripdata_Oct202021_cleaned.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (7,9) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


#Initial Investigation

In [ ]:
#investigate categorical variables
year_df.groupby(['rideable_type','member_casual']).size()

rideable_type  member_casual
classic_bike   casual           1207640
               member           1810190
docked_bike    casual            423647
               member            263383
electric_bike  casual            496005
               member            562024
dtype: int64

In [ ]:
#test for rekationship between rideable_type and member_casual
corr_year_df = year_df[['rideable_type', 'member_casual']].copy()

corr_year_df['rideable_type'] = corr_year_df['rideable_type'].astype('category').cat.codes
corr_year_df['member_casual'] = corr_year_df['member_casual'].astype('category').cat.codes

corr_year_df.corr(method = 'kendall')

,rideable_type,member_casual
rideable_type,1.000000,-0.096691
member_casual,-0.096691,1.000000


In [ ]:
#no significant correlation found

In [ ]:
#check summary statistics on numeric varables for members
year_member_df = year_df[year_df.member_casual=='member'].copy()
year_member_df = year_member_df.drop(columns=['start_lat','start_lng','end_lat','end_lng'])
year_member_df.describe()

,ride_length,day_of_week
count,2.635597e+06,2.635597e+06
mean,1.355890e+01,3.960070e+00
std,1.386693e+01,1.947018e+00
min,1.000000e+00,1.000000e+00
25%,6.020000e+00,2.000000e+00
50%,1.010000e+01,4.000000e+00
75%,1.712000e+01,6.000000e+00
max,7.052500e+02,7.000000e+00


In [ ]:
#check summary statistics on numeric variables for casual users
year_casual_df = year_df[year_df.member_casual=='casual'].copy()
year_casual_df = year_casual_df.drop(columns=['start_lat','start_lng','end_lat','end_lng'])
year_casual_df.describe()

,ride_length,day_of_week
count,2.127292e+06,2.127292e+06
mean,2.770536e+01,4.558470e+00
std,3.529045e+01,2.009488e+00
min,1.000000e+00,1.000000e+00
25%,9.820000e+00,3.000000e+00
50%,1.710000e+01,5.000000e+00
75%,3.127000e+01,6.000000e+00
max,7.083300e+02,7.000000e+00


#Routes

In [ ]:
#extract route dataframe for analysis
route_member_df = year_df[['route', 'member_casual']].copy()
route_member_df = route_member_df[route_member_df.member_casual == 'member']
route_member_df = route_member_df.groupby(['route']).size().to_frame('count')
route_member_df = route_member_df.sort_values('count', ascending = False)

route_casual_df = year_df[['route_traveled', 'member_casual']].copy()
route_casual_df = route_casual_df[route_casual_df.member_casual == 'casual']
route_casual_df = route_casual_df.groupby(['route_traveled']).size().to_frame('count')
route_casual_df = route_casual_df.sort_values('count', ascending = False)

In [ ]:
route_member_df.head()

,count
route_traveled,
Ellis Ave & 60th St to Ellis Ave & 55th St,3757
Ellis Ave & 55th St to Ellis Ave & 60th St,3387
Ellis Ave & 60th St to University Ave & 57th St,2197
University Ave & 57th St to Ellis Ave & 60th St,2152
State St & 33rd St to Calumet Ave & 33rd St,1861


In [ ]:
route_member_df['count'].describe()

count    125135.000000
mean         21.062029
std          53.876105
min           1.000000
25%           2.000000
50%           4.000000
75%          17.000000
max        3757.000000
Name: count, dtype: float64

In [ ]:
route_casual_df.head()

,count
route_traveled,
Streeter Dr & Grand Ave to Streeter Dr & Grand Ave,11272
Millennium Park to Millennium Park,6293
Michigan Ave & Oak St to Michigan Ave & Oak St,5879
Lake Shore Dr & Monroe St to Lake Shore Dr & Monroe St,5543
Buckingham Fountain to Buckingham Fountain,3632


In [ ]:
route_casual_df['count'].describe()

count    127017.000000
mean         16.748089
std          71.971438
min           1.000000
25%           1.000000
50%           4.000000
75%          13.000000
max       11272.000000
Name: count, dtype: float64

#Bike Type

In [ ]:
#extract rideable_type dataframe for analysis
ridetype_member_df = year_df[['rideable_type', 'member_casual']].copy()
ridetype_member_df = ridetype_member_df[ridetype_member_df.member_casual == 'member']
ridetype_member_df = ridetype_member_df.groupby(['rideable_type']).size().to_frame('count_member')
ridetype_member_df = ridetype_member_df.sort_values('count_member', ascending = False)
ridetype_member_df['percent_member'] = (ridetype_member_df['count_member'] / ridetype_member_df['count_member'].sum()) * 100
ridetype_member_df = ridetype_member_df.round(decimals = 2)

In [ ]:
ridetype_member_df.head()

,count_member,percent_member
rideable_type,,
classic_bike,1810190,68.68
electric_bike,562024,21.32
docked_bike,263383,9.99


In [ ]:
#investigate rideable_type for casual users
ridetype_casual_df = year_df[['rideable_type', 'member_casual']].copy()
ridetype_casual_df = ridetype_casual_df[ridetype_casual_df.member_casual == 'casual']
ridetype_casual_df = ridetype_casual_df.groupby(['rideable_type']).size().to_frame('count_casual')
ridetype_casual_df = ridetype_casual_df.sort_values('count_casual', ascending = False)
ridetype_casual_df['percent_casual'] = (ridetype_casual_df['count_casual'] / ridetype_casual_df['count_casual'].sum()) * 100
ridetype_casual_df = ridetype_casual_df.round(decimals = 2)

In [ ]:
ridetype_casual_df.head()

,count_casual,percent_casual
rideable_type,,
classic_bike,1207640,56.77
electric_bike,496005,23.32
docked_bike,423647,19.91


In [ ]:
ridetype_df = pd.merge(ridetype_casual_df, ridetype_member_df, how='left', on='rideable_type')
ridetype_df = ridetype_df.drop(columns = ['count_casual', 'count_member'])

In [ ]:
ridetype_df

,percent_casual,percent_member
rideable_type,,
classic_bike,56.77,68.68
electric_bike,23.32,21.32
docked_bike,19.91,9.99


In [ ]:
#summary statistics 
from IPython.display import display_html

ridetype_casual_describe_df = ridetype_casual_df.describe(percentiles = [.05,.25,.5,.75,.95])
ridetype_member_describe_df = ridetype_member_df.describe(percentiles = [.05,.25,.5,.75,.95])

ridetype_casual_describe_styler = ridetype_casual_describe_df.style.set_table_attributes("style='display:inline'")
ridetype_member_describe_styler = ridetype_member_describe_df.style.set_table_attributes("style='display:inline'")

display_html(ridetype_casual_describe_styler._repr_html_()+ridetype_member_describe_styler._repr_html_(), raw = True)

,count_casual,percent_casual
count,3.000000,3.000000
mean,709097.333333,33.333333
std,433263.791363,20.368236
min,423647.000000,19.910000
5%,430882.800000,20.251000
25%,459826.000000,21.615000
50%,496005.000000,23.320000
75%,851822.500000,40.045000
95%,1136476.500000,53.425000
max,1207640.000000,56.770000


In [ ]:
#summary statistics 
from IPython.display import display_html

df1_0 = ridetype_casual_df
df2_0 = ridetype_casual_df

ridetype_casual_styler = ridetype_casual_df.style.set_table_attributes("style='display:inline'")
ridetype_member_styler = ridetype_member_df.style.set_table_attributes("style='display:inline'")

display_html(ridetype_casual_styler._repr_html_()+ridetype_member_styler._repr_html_(), raw = True)


,count_casual,percent_casual
rideable_type,,
classic_bike,1207640,56.770000
electric_bike,496005,23.320000
docked_bike,423647,19.910000
,count_member,percent_member
rideable_type,,
classic_bike,1810190,68.680000
electric_bike,562024,21.320000
docked_bike,263383,9.990000


In [ ]:
ridetype_casual_df['count_casual'].sum()

2127292

In [ ]:
ridetype_member_df['count_member'].sum()

2635597

In [ ]:
(ridetype_casual_df['count_casual'].sum()/(ridetype_casual_df['count_casual'].sum() + ridetype_member_df['count_member'].sum())).round(2)

0.45

In [ ]:
#Compare this with the historic data (seems like a higher percentage than the historic data). 

#Weekday

In [ ]:
#investigate weekday for members
weekday_member_df = year_df[['day_of_week', 'member_casual']].copy()
weekday_member_df = weekday_member_df[weekday_member_df.member_casual == 'member']
weekday_member_df = weekday_member_df.groupby(['day_of_week']).size().to_frame('count_member')
weekday_member_df = weekday_member_df.sort_values('day_of_week')
weekday_member_df['percent_member'] = (weekday_member_df['count_member'] / weekday_member_df['count_member'].sum()) * 100
weekday_member_df = weekday_member_df.round(decimals = 2)

weekday_casual_df = year_df[['day_of_week', 'member_casual']].copy()
weekday_casual_df = weekday_casual_df[weekday_casual_df.member_casual == 'casual']
weekday_casual_df = weekday_casual_df.groupby(['day_of_week']).size().to_frame('count_casual')
weekday_casual_df = weekday_casual_df.sort_values('day_of_week')
weekday_casual_df['percent_casual'] = (weekday_casual_df['count_casual'] / weekday_casual_df['count_casual'].sum()) * 100
weekday_casual_df = weekday_casual_df.round(decimals = 2)

In [ ]:
weekday_df = pd.merge(weekday_casual_df, weekday_member_df, how='left', on='day_of_week')
weekday_df = weekday_df.drop(columns = ['count_casual', 'count_member'])

In [ ]:
weekday_df

,percent_casual,percent_member
day_of_week,,
1,10.93,13.32
2,10.33,14.83
3,10.56,15.38
4,11.01,14.87
5,14.20,14.63
6,23.24,14.55
7,19.73,12.43


In [ ]:
weekday_df['percent_casual'].std()

5.1856624685854005

In [ ]:
weekday_df['percent_member'].std()

1.032774810655164

In [ ]:
#Behaviors for two customer groups is significantly different. Pick this up later in visualization step.

#Stations

In [ ]:
#extract start_station_name and end_station_name dataframes for analysis
startstation_member_df = year_df[['start_station_name', 'member_casual']].copy()
startstation_member_df = startstation_member_df[startstation_member_df.member_casual == 'member']
startstation_member_df = startstation_member_df.groupby(['start_station_name']).size().to_frame('startstation_count_member')
startstation_member_df = startstation_member_df.sort_values('startstation_count_member', ascending = False)
startstation_member_df['startstation_percent_member'] = (startstation_member_df['startstation_count_member'] / startstation_member_df['startstation_count_member'].sum()) * 100
startstation_member_df = startstation_member_df.round(decimals = 2)

endstation_member_df = year_df[['end_station_name', 'member_casual']].copy()
endstation_member_df = endstation_member_df[endstation_member_df.member_casual == 'member']
endstation_member_df = endstation_member_df.groupby(['end_station_name']).size().to_frame('endstation_count_member')
endstation_member_df = endstation_member_df.sort_values('endstation_count_member', ascending = False)
endstation_member_df['endstation_percent_member'] = (endstation_member_df['endstation_count_member'] / endstation_member_df['endstation_count_member'].sum()) * 100
endstation_member_df = endstation_member_df.round(decimals = 2)

startstation_casual_df = year_df[['start_station_name', 'member_casual']].copy()
startstation_casual_df = startstation_casual_df[startstation_casual_df.member_casual == 'casual']
startstation_casual_df = startstation_casual_df.groupby(['start_station_name']).size().to_frame('startstation_count_casual')
startstation_casual_df = startstation_casual_df.sort_values('startstation_count_casual', ascending = False)
startstation_casual_df['startstation_percent_casual'] = (startstation_casual_df['startstation_count_casual'] / startstation_casual_df['startstation_count_casual'].sum()) * 100
startstation_casual_df = startstation_casual_df.round(decimals = 2)

endstation_casual_df = year_df[['end_station_name', 'member_casual']].copy()
endstation_casual_df = endstation_casual_df[endstation_casual_df.member_casual == 'casual']
endstation_casual_df = endstation_casual_df.groupby(['end_station_name']).size().to_frame('endstation_count_casual')
endstation_casual_df = endstation_casual_df.sort_values('endstation_count_casual', ascending = False)
endstation_casual_df['endstation_percent_casual'] = (endstation_casual_df['endstation_count_casual'] / endstation_casual_df['endstation_count_casual'].sum()) * 100
endstation_casual_df = endstation_casual_df.round(decimals = 2)

In [ ]:
#summary statistics 
from IPython.display import display_html

df1 = startstation_member_df.describe(percentiles = [.05,.25,.5,.75,.95])
df2 = endstation_member_df.describe(percentiles = [.05,.25,.5,.75,.95])
df3 = startstation_casual_df.describe(percentiles = [.05,.25,.5,.75,.95])
df4 = endstation_casual_df.describe(percentiles = [.05,.25,.5,.75,.95])

df1_styler = df1.style.set_table_attributes("style='display:inline'")
df2_styler = df2.style.set_table_attributes("style='display:inline'")
df3_styler = df3.style.set_table_attributes("style='display:inline'")
df4_styler = df4.style.set_table_attributes("style='display:inline'")

display_html(df1_styler._repr_html_()+df2_styler._repr_html_(), raw = True)
display_html(df3_styler._repr_html_()+df4_styler._repr_html_(), raw = True)

,startstation_count_member,startstation_percent_member
count,772.000000,772.000000
mean,3413.985751,0.129417
std,4448.286211,0.169037
min,1.000000,0.000000
5%,3.000000,0.000000
25%,45.500000,0.000000
50%,1291.000000,0.050000
75%,5648.750000,0.210000
95%,12565.100000,0.474500
max,25471.000000,0.970000


,startstation_count_casual,startstation_percent_casual
count,806.000000,806.000000
mean,2639.320099,0.123908
std,4481.810551,0.210824
min,1.000000,0.000000
5%,7.000000,0.000000
25%,107.250000,0.010000
50%,784.000000,0.040000
75%,3433.250000,0.160000
95%,10403.250000,0.490000
max,64019.000000,3.010000


In [ ]:
#Most used stations 
from IPython.display import display_html

df1_1 = startstation_member_df.head(10)
df2_1 = endstation_member_df.head(10)
df3_1 = startstation_casual_df.head(10)
df4_1 = endstation_casual_df.head(10)

df1_1_styler = df1_1.style.set_table_attributes("style='display:inline'")
df2_1_styler = df2_1.style.set_table_attributes("style='display:inline'")
df3_1_styler = df3_1.style.set_table_attributes("style='display:inline'")
df4_1_styler = df4_1.style.set_table_attributes("style='display:inline'")

display_html(df1_1_styler._repr_html_()+df2_1_styler._repr_html_(), raw = True)
display_html(df3_1_styler._repr_html_()+df4_1_styler._repr_html_(), raw = True)

,startstation_count_member,startstation_percent_member
start_station_name,,
Clark St & Elm St,25471,0.970000
Wells St & Concord Ln,23446,0.890000
Kingsbury St & Kinzie St,22616,0.860000
Wells St & Elm St,21017,0.800000
Dearborn St & Erie St,19632,0.740000
St. Clair St & Erie St,19234,0.730000
Wells St & Huron St,19203,0.730000
Broadway & Barry Ave,18619,0.710000
Clark St & Armitage Ave,17383,0.660000


,startstation_count_casual,startstation_percent_casual
start_station_name,,
Streeter Dr & Grand Ave,64019,3.010000
Millennium Park,33408,1.570000
Michigan Ave & Oak St,29799,1.400000
Shedd Aquarium,22864,1.070000
Lake Shore Dr & Monroe St,22566,1.060000
Theater on the Lake,22032,1.040000
Wells St & Concord Ln,19450,0.910000
Clark St & Lincoln Ave,17066,0.800000
Indiana Ave & Roosevelt Rd,16922,0.800000


In [ ]:
#Conclusion
# 1. Higher variability in casual user both start and end station use (to be expected)
#    - consider creating an error bar
# 2. Significant anomoly for station 'Streeter Dr & Grand Ave' for casual users. Consider further investigation.

#Ride Duration

In [ ]:
#extract ride_length dataframes for analysis
rideduration_member_df = year_df[['ride_length', 'member_casual']].copy()
rideduration_member_df = rideduration_member_df[rideduration_member_df.member_casual == 'member']
rideduration_member_df['ride_length_member'] = rideduration_member_df['ride_length']
rideduration_member_df = rideduration_member_df.drop(columns = ['ride_length'])

rideduration_casual_df = year_df[['ride_length', 'member_casual']].copy()
rideduration_casual_df = rideduration_casual_df[rideduration_casual_df.member_casual == 'casual']
rideduration_casual_df['ride_length_casual'] = rideduration_casual_df['ride_length']
rideduration_casual_df = rideduration_casual_df.drop(columns = ['ride_length'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [ ]:
#Ride Length Summary Statistics
from IPython.display import display_html

rideduration_casual_describe_df = rideduration_casual_df.describe()
rideduration_casual_describe_df = rideduration_casual_describe_df.round(2)

rideduration_member_describe_df = rideduration_member_df.describe()
rideduration_member_describe_df = rideduration_member_describe_df.round(2)

rideduration_casual_describe_styler = rideduration_casual_describe_df.style.set_table_attributes("style='display:inline'")
rideduration_member_describe_styler = rideduration_member_describe_df.style.set_table_attributes("style='display:inline'")

display_html(rideduration_casual_describe_styler._repr_html_()+rideduration_member_describe_styler._repr_html_(), raw = True)

,ride_length_casual
count,2127292.000000
mean,27.705359
std,35.290455
min,1.000000
25%,9.820000
50%,17.100000
75%,31.270000
max,708.330000
,ride_length_member
count,2635597.000000
